In [125]:
# imports

import os
import re
from math import pi
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, Div, LabelSet
from bokeh.plotting import figure
from bokeh.transform import dodge, cumsum
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [18]:
def preprocess_intake_files(subject):
    '''
    Function that preprocesses the intakes files from the given subject to the correct formattings
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    
    Returns
    --------------
    list
        baseline_filename    the filename of the created baseline intake file
        vegan_filename       the filename of the created vegan intake file
    '''
    
    days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    baseline = ["week1", "week2"]
    header = "Day,Date,Year,Calories,Fat,Saturated fat,Carbs,Fiber,Sugar,Protein,Sodium,Cholesterol,Potassium\n"
    
    food_intake_files_path = '../data/intake_files'
    processed_files_path = food_intake_files_path + '/processed'
    
    baseline_filename = processed_files_path + '/nutrition_baseline_subject_'+ subject +'.csv'
    nutrition_baseline = open(baseline_filename, 'w')
    nutrition_baseline.write(header)
    
    vegan_filename = processed_files_path + '/nutrition_vegan_subject_'+ subject +'.csv'
    nutrition_vegan = open(vegan_filename, 'w')
    nutrition_vegan.write(header)
    
    with nutrition_baseline as n_baseline, nutrition_vegan as n_vegan:
        for file in os.scandir(food_intake_files_path):
            filename = file.path
            # only use the file from the given subject
            if (('subject_' + subject in filename) and filename.lower().endswith('.csv')):
                file = open(file, 'r')
                with file as food_intake:
                    # start at line 10 with reading the food log
                    for i in range(9):
                        food_intake, next(food_intake)
                    for line in food_intake:
                        # replace double quotes with single quotes
                        index = 1 if '""' in line else 0
                        line = line.strip().replace('""', '"')[index:].rstrip('"')
                        # get the nutrition details per day
                        if line.split(',')[0].strip('"').lower() in days:
                            if any(week in filename.lower() for week in baseline):
                                n_baseline.write(line + '\n')
                            else:
                                n_vegan.write(line + '\n')
    
    return [baseline_filename, vegan_filename]

def create_nutrition_tables(subject):
    '''
    Function that creates the food intake data tables for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    
    Returns
    --------------
    list
        nutrition_data    food intake data table
    '''
    
    nutrition_figures = []
    
    subject_data = preprocess_intake_files(subject)
    base_data = pd.read_csv(subject_data[0], sep=',')
    vegan_data = pd.read_csv(subject_data[1], sep=',')

    convert_columns = ['Calories', 'Fat', 'Saturated fat', 'Carbs', 
                       'Fiber', 'Sugar', 'Protein', 
                       'Sodium', 'Cholesterol', 'Potassium']

    # convert commas to points as delimiter and convert to numeric values
    base_data[convert_columns] = base_data[convert_columns].replace(',', '.', regex=True)
    base_data[convert_columns] = base_data[convert_columns].apply(pd.to_numeric, errors='coerce')
    vegan_data[convert_columns] = vegan_data[convert_columns].replace(',', '.', regex=True)
    vegan_data[convert_columns] = vegan_data[convert_columns].apply(pd.to_numeric, errors='coerce')

    # convert sodium, cholesterol and potassium to grams (they are initially in mg)
    base_data[convert_columns[7:]] = base_data[convert_columns[7:]] / 100
    vegan_data[convert_columns[7:]] = vegan_data[convert_columns[7:]] / 100
    
    # set the average saturated fat of the week to '0' if it is higher than the fat value
    base_data.loc[(base_data['Saturated fat'] > base_data['Fat']), 'Saturated fat'] = 0
    vegan_data.loc[(vegan_data['Saturated fat'] > vegan_data['Fat']), 'Saturated fat'] = 0
    
    # add the diet definition column
    base_data['Diet'] = 'Normal'
    vegan_data['Diet'] = 'Vegan'
    
    # concat diets and set date as index column
    nutrition_data = pd.concat([base_data, vegan_data])
    nutrition_data['Date'] = nutrition_data['Date'].astype(str) + ' ' +nutrition_data['Year'].astype(str)
    nutrition_data['Date'] = pd.to_datetime(nutrition_data['Date'], format=' %B %d %Y')
    nutrition_data.drop(['Year'], inplace=True, axis=1)
    nutrition_data.set_index('Date', inplace=True)
    nutrition_data.sort_index(inplace=True)
    
    return nutrition_data

def create_average_nutrition_figure(subject, nutrition_data):
    '''
    Function that creates the nutrition graph for the given subject
    
    Parameters
    --------------
    subject : chr
        The given subject's initial
    nutrition_data : DataFrame
        The food intake data table of the given subject
    
    Returns
    --------------
    figure
        The created nutrition graph visualization
    '''
    
    # calcuate the average nutritional values
    averages = nutrition_data.groupby(['Diet']).mean().round(2)
    
    bar_labels = ['Carbs', 'Protein', 'Fat', 
                 'Saturated fat', 'Fiber', 'Sugar', 
                 'Sodium', 'Cholesterol', 'Potassium']
    data_obj = {
            'labels': bar_labels,
            'Normal': averages[bar_labels].loc['Normal'],
            'Vegan': averages[bar_labels].loc['Vegan'],
            'None' : [0 for i in bar_labels]
        }
    source = ColumnDataSource(data=data_obj)
    
    p = figure(x_range=bar_labels, y_range=(0, 400), plot_height=400,
               title="Average nutritional information subject " + subject + " in grams",
               toolbar_location=None, tools="")
    
    p.vbar(x=dodge('labels', -0.18, range=p.x_range), 
           top='Normal', name='Normal', width=0.3, 
           source=source, color="#abdfff", legend_label="Normal diet", line_color="#75cbff", line_width=0.4)

    # middle label
    p.vbar(x=dodge('labels',  0,  range=p.x_range), top='None', width=0.1, source=source)
    
    p.vbar(x=dodge('labels',  0.18,  range=p.x_range), 
           top='Vegan', name='Vegan', width=0.3, 
           source=source, color="#ceffc4", legend_label="Vegan diet", line_color="#8dff75", line_width=0.4)
    
    p.x_range.range_padding = 0.05
    p.xgrid.grid_line_color = None
    p.legend.location = "top_right"
    p.legend.orientation = "vertical"
    p.xaxis.major_label_orientation = 0.5
    
    p.add_tools(HoverTool(
        names = ['Normal', 'Vegan'],
        tooltips = [
            ('', '$name diet'),
            ('', '@$name{1.11} grams')
        ],
        mode = 'mouse',
        show_arrow = False,
        point_policy = 'follow_mouse'
    ))
    
    style = {
        'font-size': 'smaller'
    }
    div = Div(text=
              """Average calories:<br>
                 Normal diet: {} kcal <br>
                 Vegan diet:  {} kcal"""
              .format(averages['Calories'].loc['Normal'], averages['Calories'].loc['Vegan']),
              width=200, height=100, style=style)
    show(p)
    show(div)
    return p

In [130]:
def create_nutrition_piecharts(subject, nutrition_data):
    '''
    
    '''
    weekly_averages = get_weekly_averages(subject, nutrition_data)
    macros = weekly_averages[['Carbs', 'Protein', 'Fat']].iloc[0]
    macro_colors = ['#266298', '#89c609', '#fc2a35']
    R = 0.35
    
    data = (pd.Series(macros.to_dict())
            .reset_index(name='value')
            .rename(columns={'index':'macro'})
            .assign(end_angle=lambda d: np.cumsum(d['value'] / d['value'].sum() * 2 * pi),
                    start_angle=lambda d: np.pad(d['end_angle'], (1, 0))[:-1],
                    label_x=lambda d: R * 0.95 * np.cos(d['start_angle']),
                    label_y=lambda d: R * 0.95 * np.sin(d['start_angle'])))
    
    data['percentage'] = (data['value']/data['value'].sum() * 100).round(2).astype(str) + '%'
    data['color'] = macro_colors[:len(macros)]

    p_range = (-R * 1.2, R * 1.2)
    p_size = 500
    p = figure(title="Pie Chart", toolbar_location=None,
               plot_height=p_size, plot_width=p_size,
               x_range=p_range, y_range=p_range,
               tools="")

    p.wedge(x=0, y=0, radius=R,
            start_angle='start_angle', end_angle='end_angle',
            line_color="white", fill_color='color', legend_field='macro', source=data)

    source = ColumnDataSource(data)
    labels = LabelSet(x='label_x', y='label_y', text='percentage',
                      angle='start_angle',
                      source=source, render_mode='canvas', text_align='right')

    p.add_layout(labels)
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None

    show(p)
    
def get_weekly_averages(subject, nutrition_data):
    nutrition_data['week_number'] = ((nutrition_data.Day.str.lower() == 'monday').cumsum())
    return nutrition_data.groupby('week_number').mean().round(2)
    

In [131]:
def create_nutrition_graphs(subjects):
    for subject in subjects:
        nutrition_data = create_nutrition_tables(subject)
        
        nutrition_figures = []
        #figure = create_average_nutrition_figure(subject, nutrition_data)
        #nutrition_figures.append(figure)
        
    create_nutrition_piecharts(subjects[:-1], nutrition_data)

def main():
    subjects = ['A', 'B', 'C', 'D', 'E']
    create_nutrition_graphs(subjects)

if __name__ == '__main__':
    main()